In [1]:
p=17
N=8
k=4
m=N//k
print("Parameters: \n  p:{}\n  N:{}\n  k:{}\n".format(p,N,k))

print("Domains")
ff=GF(p)

A=PolynomialRing(ff,['x'])
Aq=A.quotient(A("x^{}-1".format(N)))

B=PolynomialRing(A,['y'])
Bq_base=PolynomialRing(Aq,['y'])
Bq=Bq_base.quotient(Bq_base("y^{}-1".format(m)))

C=PolynomialRing(ff,['x','y'])
Cq=C.quotient(C("x^{}-1".format(N))).quotient(C("y^{}-1".format(m)))

print("A")
print(A)
print("B")
print(B)
print("C")
print(C)
print()

A_x=A('x')
B_y=B('y')
C_x=C('x')
C_y=C('y')


print("Aq")
print(Aq)
print("Bq")
print(Bq)
print("Cq")
print(Cq)

Aq_x=Aq('x')
Bq_y=Bq('y')
Cq_x=Cq('x')
Cq_y=Cq('y')


#X.hom([])
#pr_q_wrapped = PolynomialRing(pr,'y')
#pr2

Parameters: 
  p:17
  N:8
  k:4

Domains
A
Univariate Polynomial Ring in x over Finite Field of size 17
B
Univariate Polynomial Ring in y over Univariate Polynomial Ring in x over Finite Field of size 17
C
Multivariate Polynomial Ring in x, y over Finite Field of size 17

Aq
Univariate Quotient Polynomial Ring in xbar over Finite Field of size 17 with modulus x^8 + 16
Bq
Univariate Quotient Polynomial Ring in ybar over Univariate Quotient Polynomial Ring in xbar over Finite Field of size 17 with modulus x^8 + 16 with modulus y^2 + 16
Cq
Quotient of Multivariate Polynomial Ring in x, y over Finite Field of size 17 by the ideal (y^2 - 1, x^8 - 1)


In [2]:
type(ff)

<class 'sage.rings.finite_rings.finite_field_prime_modn.FiniteField_prime_modn_with_category'>

In [3]:
print("Morphisms:")
        
C_to_A = C.hom([A_x,A_x^m])

print("C->A")
print(C_to_A)
c=C_x+C_y+1
print(c)
print("becomes")
c_a=C_to_A(c)
print(c_a)
print()

# B_to_A = B.hom([A_x^m])
# print("B->A")
# print(B_to_A)

Morphisms:
C->A
Ring morphism:
  From: Multivariate Polynomial Ring in x, y over Finite Field of size 17
  To:   Univariate Polynomial Ring in x over Finite Field of size 17
  Defn: x |--> x
        y |--> x^2
x + y + 1
becomes
x^2 + x + 1



In [4]:
A_to_C=A.hom([C_x])
print("A->C")
print(A_to_C)
a=A_x+1
print(a)
print("becomes")
print(A_to_C(a))

A->C
Ring morphism:
  From: Univariate Polynomial Ring in x over Finite Field of size 17
  To:   Multivariate Polynomial Ring in x, y over Finite Field of size 17
  Defn: x |--> x
x + 1
becomes
x + 1


In [5]:
a1=A_x^2+A_x+1
a2=2*A_x^2+2*A_x+2
b=a1*B_y+a2
B.base_ring()
print([A_to_C(b[i]) for i in range(N)])

[2*x^2 + 2*x + 2, x^2 + x + 1, 0, 0, 0, 0, 0, 0]


In [6]:
class B_to_C_morph(Morphism):
    def __init__(self,B,C,maps):
        Morphism.__init__(self,Hom(B,C))
        self.xmap=maps[0]
        self.ymap=maps[1]
    def _call_(self,x):
        return sum(self.xmap(x[i])*self.ymap^i for i in range(x.degree()))

print("B->C")
B_to_C=B_to_C_morph(B,C,[A_to_C,C_y])
print(B_to_C)
a1=A_x^2+A_x+1
a2=2*A_x^2+2*A_x+2
a3=3*A_x^2+3*A_x+3
b=a3*B_y^2+a2*B_y+a1
print(b)
print("becomes")
b_c = B_to_C(b)
print(b_c)

B->C
Generic morphism:
  From: Univariate Polynomial Ring in y over Univariate Polynomial Ring in x over Finite Field of size 17
  To:   Multivariate Polynomial Ring in x, y over Finite Field of size 17
(3*x^2 + 3*x + 3)*y^2 + (2*x^2 + 2*x + 2)*y + x^2 + x + 1
becomes
2*x^2*y + x^2 + 2*x*y + x + 2*y + 1


In [7]:
[[b_c[i,j] for i in range(3)] for j in range(3) ]

[[1, 1, 1], [2, 2, 2], [0, 0, 0]]

In [8]:
C_to_A(B_to_C(b))

2*x^4 + 2*x^3 + 3*x^2 + x + 1

In [9]:
a1.degree()

2

In [10]:
class A_to_B_morph(Morphism):
    def __init__(self,k,A,B):
        Morphism.__init__(self,Hom(A,B))
        self.B_y=self.codomain().an_element()
        self.A_x=self.domain().an_element()
        self.k=k
    def _call_(self,x):
        #A_to_Bx = A.hom([self.B_x])
        N=x.degree()+1
        assert (N%self.k==0), "{} is not divisible by {}".format(N,self.k)
        reduced = sum(sum(x[N//self.k*j+i]*self.A_x^(i) for i in range(N//self.k)) * self.B_y^j for j in range(self.k))
        return reduced
A_to_B=A_to_B_morph(3,A,B)
print("A_to_B")
print(A_to_B)
aN = a2*A_x^3+a1
print(aN)
b=A_to_B(aN)
print("becomes")
print(b)

A_to_B
Generic morphism:
  From: Univariate Polynomial Ring in x over Finite Field of size 17
  To:   Univariate Polynomial Ring in y over Univariate Polynomial Ring in x over Finite Field of size 17
2*x^5 + 2*x^4 + 2*x^3 + x^2 + x + 1
becomes
(2*x + 2)*y^2 + (2*x + 1)*y + x + 1


AttributeError: 'FiniteField_prime_modn_with_category' object has no attribute 'pari_bnf'